In [1]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [2]:
from keras.layers import *
from keras.models import Model
import tensorflow as tf

In [8]:
### Submodel for CRNN 
#CNN Size start from 2
_num_CNNkernalSize = 3

##Here is the sub model for timeDistribute layer in main model

# the share layer for all time distribution CNN
Lines_Input = Input(name= 'TextInput',shape=(20, 100, 1)) #(None,20,100,1)
LineFeature_List = []
for i in range(_num_CNNkernalSize):
    #result_shape = (timestep, sentencelen(num of CNN result for each kernal), WE_dim, num_kernal) = (None,20,100,32)
    CL  = Conv2D(32, kernel_size = (i+2,100), padding='same', activation = 'relu')(Lines_Input) 
    #max pooling for every kernal, result_shape = (None, 1, 1, 32)
    MPL = MaxPooling2D(20,100)(CL)
    MP_shape =  tf.shape(MPL)
    #flatten for matching GRU input size
    FL  = Flatten()(MPL) #result = (timestep size , )
    LineFeature_List.append(FL)
    
LineFeature = concatenate(LineFeature_List)
TextCNNMode = Model(inputs = [Lines_Input], outputs = LineFeature)

In [13]:
###Main model 
#Lyric feature extraction, CRNN part
Text_Input = Input(name='LyricInput', shape=(None,20, 100, 1)) #(batch_size, timestep, 20, 100 ,1)
TD_CNN = TimeDistributed(TextCNNMode)(Text_Input)
GRU_F = GRU(_num_CNNkernalSize*32, return_sequences=False, kernel_initializer='he_normal', name='GRU_forward')(TD_CNN)
GRU_B = GRU(_num_CNNkernalSize*32, return_sequences=False, kernel_initializer='he_normal', name='GRU_backward', go_backwards=True)(TD_CNN)
LF = concatenate([GRU_F, GRU_B])

#pseudo-model for sheck summary
#Softmax = Dense(50, kernel_initializer='he_normal',activation='softmax')(LF)
#text_model = Model(inputs=[Text_Input], outputs=Softmax)

In [12]:
text_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
LyricInput (InputLayer)         (None, None, 20, 100 0                                            
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, None, 96)     28896       LyricInput[0][0]                 
__________________________________________________________________________________________________
GRU_forward (GRU)               (None, 96)           55584       time_distributed_3[0][0]         
__________________________________________________________________________________________________
GRU_backward (GRU)              (None, 96)           55584       time_distributed_3[0][0]         
__________________________________________________________________________________________________
concatenat

In [17]:
###Main model 
#Audio feature extraction, FCN part
_mel_scale = 96
_time_len = 1292
_channels = 1

Spectrogram_Input = Input(name= 'AudioInput',shape=(_mel_scale, _time_len, _channels)) #(None(1), 96, 1292, 1)
CL_1 = Conv2D(128, kernel_size=(3, 3),padding='same', activation = 'relu', name = "CL_1")(Spectrogram_Input) #(None, 96, 1292, 128) 
BN_1 = BatchNormalization()(CL_1)
MP_1 = MaxPool2D(pool_size= (2,4), name = "MPL_1")(BN_1) #(None, 48, 323, 128)
DP_1 = Dropout(0.5)(MP_1)

CL_2 = Conv2D(384, kernel_size=(3, 3),padding='same', activation = 'relu', name = "CL_2")(DP_1) # (None, 48, 323, 384) 
BN_2 = BatchNormalization()(CL_2)
MP_2 = MaxPool2D(pool_size= (4,5), name = "MPL_2")(BN_2) # (None, 12, 64, 384) 
DP_2 = Dropout(0.5)(MP_2)

CL_3 = Conv2D(768, kernel_size=(3, 3),padding='same', activation = 'relu', name = "CL_3")(DP_2) #(None, 12, 64, 768) 
BN_3 = BatchNormalization()(CL_3)
MP_3 = MaxPool2D(pool_size= (3,8), name = "MPL_3")(BN_3) #(None, 4, 8, 768)
DP_3 = Dropout(0.5)(MP_3)

CL_4 = Conv2D(2048, kernel_size=(3, 3),padding='same', activation = 'relu', name = "CL_4")(DP_3) #(None, 4, 8, 2048) 
BN_4 = BatchNormalization()(CL_4)
MP_4 = MaxPool2D(pool_size= (4,8), name = "MPL_4")(BN_4)#(None, 1, 1, 2048) 
DP_4 = Dropout(0.5)(MP_4)

AF = Flatten()(DP_4) # (None, 2048) 
#pseudo-model for sheck summary
#Softmax = Dense(50, kernel_initializer='he_normal',activation='softmax')(AF)
#audio_model = Model(inputs=[Spectrogram_Input], outputs=Softmax)

In [16]:
audio_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
AudioInput (InputLayer)      (None, 96, 1292, 1)       0         
_________________________________________________________________
CL_1 (Conv2D)                (None, 96, 1292, 128)     1280      
_________________________________________________________________
batch_normalization_5 (Batch (None, 96, 1292, 128)     512       
_________________________________________________________________
MPL_1 (MaxPooling2D)         (None, 48, 323, 128)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 48, 323, 128)      0         
_________________________________________________________________
CL_2 (Conv2D)                (None, 48, 323, 384)      442752    
_________________________________________________________________
batch_normalization_6 (Batch (None, 48, 323, 384)      1536      
__________

Total params: 17,372,594
Trainable params: 17,365,938
Non-trainable params: 6,656
_____________________________

In [18]:
#Main model
_num_Tags = 50
_num_neural = 2

SF = concatenate([AF,LF]) # Song feature for all song

Output_Layers = []
# add MLT classifer
for i in range(_num_Tags):
    MTL_L = Dense(_num_neural, activation='sigmoid', name='Tag_'+str(i+1))(SF)
    Output_Layers.append(MTL_L)

Main_model = Model(inputs=[Text_Input,Spectrogram_Input], outputs=Output_Layers)

In [19]:
Main_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
AudioInput (InputLayer)         (None, 96, 1292, 1)  0                                            
__________________________________________________________________________________________________
CL_1 (Conv2D)                   (None, 96, 1292, 128 1280        AudioInput[0][0]                 
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (None, 96, 1292, 128 512         CL_1[0][0]                       
__________________________________________________________________________________________________
MPL_1 (MaxPooling2D)            (None, 48, 323, 128) 0           batch_normalization_9[0][0]      
__________________________________________________________________________________________________
dropout_9 

In [24]:
import os 
os_AudioSpectrogram = './Spectrogram'
os_LineMatrix = './LineMatrix'

In [25]:
os_LineMatrix

'./LineMatrix'

In [26]:
import json 
import codecs
from sklearn.model_selection import train_test_split

with codecs.open("VectorizeTagTraing_50d_Dic.json",'r',encoding = 'utf8') as infile:
    Tag_Data = json.load(infile)
Data_Set = [[x.replace('.json',''), Tag_Data[x.replace('.json','')]] for x in os.listdir(os_LineMatrix)]
x_Text_List = np.array([x[0] for x in Data_Set])
y_Text_List = np.array([y[1] for y in Data_Set])
x_train , x_test, y_train, y_test = train_test_split(x_Text_List, y_Text_List, test_size=0.2, random_state=100)

In [27]:
from keras.utils import Sequence
import json
import codecs

class LyricDataGenerator(Sequence):
    #batch size can only be 1
    def __init__(self, text_filenames, labels):
        self.text_filenames, self.labels = text_filenames, labels

    def __len__(self):
        return len(self.text_filenames)

    def __getitem__(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        filename = self.text_filenames[idx]
        with codecs.open(os.path.join(os_LineMatrix,filename),'r',encoding = 'utf8') as infile:
            batch_x = np.array(json.load(infile))
        batch_x = batch_x.reshape(len(batch_x),20,100,1)
        
        batch_y = self.labels[idx]

        return np.array([batch_x]), np.array([batch_y])
    def getitem(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        
        return self.__getitem__(idx)

In [28]:
testLDG = LyricDataGenerator(x_train,y_train)

In [29]:
LDG_x , LDG_y = testLDG.getitem(0)

In [30]:
print(LDG_x.shape)
print(LDG_y.shape)

(1, 40, 20, 100, 1)
(1, 50)


In [31]:
class AudioDataGenerator(Sequence):
    #batch size can only be 1
    def __init__(self, text_filenames, labels):
        self.text_filenames, self.labels = text_filenames, labels

    def __len__(self):
        return len(self.text_filenames)

    def __getitem__(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        filename = self.text_filenames[idx]
        with codecs.open(os.path.join(os_AudioSpectrogram,filename),'r',encoding = 'utf8') as infile:
            batch_x = np.array(json.load(infile)).reshape(96, 1292, 1)
        
        batch_y = self.labels[idx]

        return np.array([batch_x]), np.array([batch_y])
    def getitem(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        
        return self.__getitem__(idx)

In [32]:
class SongDataGenerator(Sequence):
    #batch size can only be 1
    def __init__(self, text_filenames, labels):
        self.text_filenames, self.labels = text_filenames, labels

    def __len__(self):
        return len(self.text_filenames)

    def __getitem__(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        filename = self.text_filenames[idx]
        
        with codecs.open(os.path.join(os_LineMatrix,filename),'r',encoding = 'utf8') as infile:
            Text_batch_x = np.array(json.load(infile))
        Text_batch_x = Text_batch_x.reshape(len(Text_batch_x),20,100,1) #(num_sentence, 20 word ,100dim WE, 1for cnn)
        
        with codecs.open(os.path.join(os_AudioSpectrogram,filename),'r',encoding = 'utf8') as infile:
            Audio_batch_x = np.array(json.load(infile)).reshape(96, 1292, 1) #(96mel scale , 1292 time window)
        
        MLT_y = []
        
        batch_y = self.labels[idx]
        for gt in batch_y:
            binary_gt = 0 if gt == 1 else 1
            MLT_y.append(np.array([[gt, binary_gt]]))
        
        
        return [np.array([Text_batch_x]), np.array([Audio_batch_x])], MLT_y
    def getitem(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        
        return self.__getitem__(idx)

In [33]:
SDG = SongDataGenerator(x_train,y_train)

In [34]:
SDG_x, SDG_y = SDG.getitem(0)

In [35]:
SDG_x[0].shape

(1, 40, 20, 100, 1)

In [36]:
SDG_y[0].shape

(1, 2)

In [37]:
SDG_x[1].shape

(1, 96, 1292, 1)

In [38]:
SDG_train = SongDataGenerator(x_train, y_train)
SDG_test = SongDataGenerator(x_test, y_test)

In [39]:
epochs = 10

In [40]:
Main_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
Main_model.fit_generator(generator=SDG_train,
                    #steps_per_epoch=(num_training_samples // batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=SDG_test,                      
                    #validation_steps=(num_validation_samples // batch_size),
                    )

Epoch 1/10
